In [75]:
import numpy as np
import pandas as pd
%matplotlib inline 
from collections import Counter

import time
from scipy.spatial.distance import euclidean
from scipy import stats
from matplotlib import pyplot as plt

# Завантажуємо та фільтруємо дані

In [76]:
frame = pd.read_csv('diamonds.csv')
frame.drop('Unnamed: 0', axis = 1, inplace = True)

In [77]:
cool = {'Fair':1, 'Good':2, 'Very Good':3, 'Premium':4, 'Ideal':5}
frame.cut = frame.cut.map(cool)
frame.head(3)

points = np.array(frame[['price','carat']][0:150], dtype=float)
labels = np.array(frame['cut'][0:150],dtype=float)

# Головна частина

In [78]:
def sort_dist(points):
    # знаходимо дистанцію до інших елементів від поточної точки 

    dists = np.array([[ [ euclidean(points[i],points[j]), j ] for j in range(points.shape[0])]
                for i in range( points.shape[0])] , dtype=float)
    
    # упорядковуємо по зростанню відстанні до і-го елемента
    for i in range(len(dists)):
        dists[i] = dists[i][dists[i][:,0].argsort()]
        
    return dists

# Функція, яка дозволить нам вибрати оптимальне значення k

In [79]:
def trein_kNN(trein_set, label_set):
    trein_set = np.array(trein_set)
    label_set = np.array(label_set)

    # знаходимо дистанцію до інших елементів від поточного і далі сортуємо по збільшенні відстанні між елементам   
    dist = sort_dist(trein_set)
    
    k_result = {}
    # намагаємося вибрати оптимальне k
    for k in np.arange(1,len(trein_set)):
        true_recogn = 0
        
        for i in range(len(trein_set)):
            class_knn = [] 
            
            # добавляємо в список класи k ближніх об'єктів
            for j in np.arange(1,(k+1)):
                class_knn.append(label_set[int(dist[i,j,1])])
                
            # знаходимо клас який максимально часто зустрічається
            count_class = Counter(class_knn)
            
            # знаходимо класс елемента
            target_class = max(count_class, key=lambda key:count_class[key])
            
            # якщо відповідь на k-елементі правильна, то результативність збільшуємо на одиницю
            if(target_class == label_set[i]):
                true_recogn += 1
                
        k_result[k] = true_recogn
    
    # знаходимо оптимальне значення k
    target_k = max(k_result, key=lambda key: k_result[key])
    
    return target_k, k_result

In [80]:
target_k, k_result = trein_kNN(points, labels)
print(" кількість правильних передбачень при кількості сусідів k: \n \n", k_result)
print()
print('k у якого кількість правильних передбачень найбільша: \nk = ',target_k)

 кількість правильних передбачень при кількості сусідів k: 
 
 {1: 72, 2: 72, 3: 78, 4: 70, 5: 72, 6: 73, 7: 65, 8: 66, 9: 64, 10: 61, 11: 63, 12: 59, 13: 59, 14: 53, 15: 55, 16: 55, 17: 55, 18: 54, 19: 58, 20: 57, 21: 56, 22: 60, 23: 57, 24: 61, 25: 61, 26: 60, 27: 62, 28: 63, 29: 61, 30: 64, 31: 64, 32: 56, 33: 56, 34: 56, 35: 55, 36: 55, 37: 58, 38: 58, 39: 58, 40: 58, 41: 58, 42: 59, 43: 59, 44: 59, 45: 58, 46: 58, 47: 58, 48: 58, 49: 58, 50: 58, 51: 58, 52: 58, 53: 58, 54: 58, 55: 58, 56: 58, 57: 58, 58: 58, 59: 58, 60: 58, 61: 58, 62: 58, 63: 58, 64: 58, 65: 58, 66: 58, 67: 58, 68: 58, 69: 58, 70: 58, 71: 58, 72: 58, 73: 58, 74: 54, 75: 54, 76: 54, 77: 54, 78: 54, 79: 58, 80: 54, 81: 54, 82: 58, 83: 54, 84: 54, 85: 54, 86: 58, 87: 58, 88: 58, 89: 58, 90: 58, 91: 58, 92: 58, 93: 58, 94: 58, 95: 58, 96: 58, 97: 58, 98: 54, 99: 58, 100: 58, 101: 58, 102: 58, 103: 58, 104: 58, 105: 58, 106: 58, 107: 58, 108: 58, 109: 58, 110: 58, 111: 58, 112: 58, 113: 58, 114: 54, 115: 42, 116: 47, 

# Функція для передбачення класу невідомої точки

In [81]:
def dist_to_point(points,unknown):
    
    # знаходимо дистанцію до інших елементів від поточної точки 
    
    num_pred = unknown.shape[0]
    num_data = points.shape[0]
    dists = np.zeros((num_pred,num_data))
    
    for i in range(num_pred):
        for j in range(num_data):
            dists[i,j] = euclidean(unknown[i],points[j])
            
    return dists 


In [82]:
def kNN(data_x,class_x, unknown,k=2):
    data_x = np.array(data_x)
    class_x = np.array(class_x)  #.reshape((len(class_x),1))
    unknown = np.array(unknown)

        
    # знаходимо дистанцію до інших елементів від поточного і далі сортуємо по збільшенні відстанні між елементам   
    dists = dist_to_point(data_x,unknown)
    
    num_pred = dists.shape[0]
    results = np.zeros(num_pred)
    
    
    for i in range(num_pred):
        dst = dists[i]
        closest_y = class_x[dst.argsort()[:k]]
        results[i] = Counter(closest_y).most_common(1)[0][0]
    
    return results

#  перевіряємо роботу алгоритма

In [83]:
points = np.array(frame[['price','carat']][0:100], dtype=float)
labels = np.array(frame['cut'][0:100],dtype=float)
target = np.array(frame[['price','carat']][80:90], dtype=float)

target_k, k_result = trein_kNN(points, labels)

In [84]:
print('k = ',target_k)
results = kNN(points, labels, target,target_k)
print(results)

k =  3
[ 3.  3.  3.  5.  3.  4.  4.  4.  4.  4.]


In [85]:
labels_s = np.array(frame['cut'][80:90],dtype=float)
print(labels_s)

[ 3.  3.  5.  5.  2.  4.  4.  4.  4.  4.]


# Порівнянння із sklearn

In [95]:
from sklearn.neighbors import NearestNeighbors
from sklearn import neighbors

In [96]:
X = np.array(frame[['price','carat']][0:100], dtype=float)
Y = np.array(frame['cut'][0:100],dtype=float)
target = np.array(frame[['price','carat']][80:90], dtype=float)


nbrs =  neighbors.KNeighborsClassifier()
nbrs.fit(X,Y)

distances, indices = nbrs.kneighbors(X)

Z = nbrs.predict(target)
print(Z)

[ 3.  3.  3.  4.  3.  4.  4.  4.  4.  4.]
